In [1]:
using Revise
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using VMRobotControl: remake
using DifferentialEquations
using MeshIO
include("../../functions.jl")

display_transform (generic function with 1 method)

## Creating a simple mechanism for test purposes

In [2]:
mechanism = Mechanism{Float64}("TestRobot")
F0 = root_frame(mechanism)
F1 = add_frame!(mechanism; id="L1_frame")
F2 = add_frame!(mechanism; id="L2_frame")

# Define the unit vectors to make things easier
X = SVector(1., 0., 0.)
Y = SVector(0., 1., 0.)
Z = SVector(0., 0., 1.)

J1 = Prismatic(X)
J2 = Prismatic(Y)
add_joint!(mechanism, J1; parent="root_frame", child="L1_frame", id="J1")
add_joint!(mechanism, J2; parent="L1_frame", child="L2_frame", id="J2")

add_coordinate!(mechanism, FrameOrigin(F0); id="base_centre_of_mass")
add_coordinate!(mechanism, FrameOrigin(F1); id="l1_centre_of_mass")
add_coordinate!(mechanism, FrameOrigin(F2); id="l2_centre_of_mass")

add_component!(mechanism, PointMass(1.0, "base_centre_of_mass"); id="base_mass")
add_component!(mechanism, PointMass(1.0, "l1_centre_of_mass"); id="l1_mass")
add_component!(mechanism, PointMass(1.0, "l2_centre_of_mass"); id="l2_mass")

I_mat = @SMatrix [
    0.01  0.    0.  ;
    0.    0.01  0.  ;
    0.    0.    0.01
]

add_inertia!(mechanism, F1, I_mat; id="L1_inertia")
add_inertia!(mechanism, F2, I_mat; id="L2_inertia")

add_coordinate!(mechanism, JointSubspace("J1"); id="J1")
add_component!(mechanism, LinearDamper(0.5, "J1"); id="J1_damper")
add_coordinate!(mechanism, JointSubspace("J2"); id="J2")
add_component!(mechanism, LinearDamper(0.5, "J2"); id="J2_damper")
add_gravity_compensation!(mechanism, VMRobotControl.DEFAULT_GRAVITY)

### Visualization

In [3]:
m = compile(mechanism)
kcache = Observable(new_kinematics_cache(m))

# Setup the figure
fig1 = Figure(size=(700, 350))
ls = LScene(fig1[1, 1]; show_axis=false)

# Or a sketch of it's kinematic structure
robotsketch!(ls, kcache; scale=0.3, linewidth=2.5, transparency=true)

display(fig1)


GLMakie.Screen(...)

## Introducing Virtual Mechanisms

In [4]:
vms = VirtualMechanismSystem("testVMS", mechanism)
root = root_frame(vms.robot)

refpos = -0.5*X -0.5*Y
add_coordinate!(vms, FrameOrigin(".robot.L2_frame"); id="ee_position")
add_coordinate!(vms, ConstCoord(refpos); id="ref")
add_coordinate!(vms, CoordDifference("ee_position", "ref"); id="ee_error")
add_coordinate!(vms, FramePoint(".robot.$root", SVector(0.0, -0.075, 0.32));  id="Target position")

ee_stiffness = SMatrix{3, 3}(100.0, 0.0, 0.0, 0.0, 100.0, 0.0, 0.0, 0.0, 100.0)
ee_damping = SMatrix{3, 3}(15.0, 0.0, 0.0, 0.0, 15.0, 0.0, 0.0, 0.0, 15.0)

add_component!(vms, LinearSpring(ee_stiffness, "ee_error"); id="ee_spring")
add_component!(vms, LinearDamper(ee_damping, "ee_error"); id="ee_damper")

"ee_damper"

## Simulating the Robot

### Setting Up the Simulation

In [5]:
phase_2 = false

f_setup(cache) = return (get_compiled_coordID(cache, "ref"), phase_2)

function f_control(cache, t, args, extra)
    ref_pos_coord_id, phase_2 = args
    
    if t > 3 && !phase_2
        cache[ref_pos_coord_id] = remake(cache[ref_pos_coord_id]; coord_data=ConstCoord(-1*X))
        phase_2 = true
    end 

    nothing
end

tspan = (0., 5)
vms_compiled = compile(vms)
q = (zero_q(vms_compiled.robot), Float64[]) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), Float64[]) # Robot joint velocity, vm joint velocities
g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)

using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

sol = solve(prob, Tsit5(), progress=true; maxiters=1e6, abstol=1e-6, reltol=1e-6);

ODE   0%|█                                              |  ETA: N/A
ODE 100%|███████████████████████████████████████████████| Time: 0:00:03


### Visualizing the Results

In [7]:
dcache = new_dynamics_cache(m)
display(fig1)
animate_robot_odesolution(fig1, sol, kcache, "robot_animation.mp4")

"robot_animation.mp4"

In [19]:
display(sol)

retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 103-element Vector{Float64}:
 0.0
 9.999999999999999e-5
 0.0010999999999999998
 0.005484202058669556
 0.012478169895907838
 0.02087539187631083
 0.03173402139454368
 0.0446438771468398
 0.06006057789123498
 0.0775481542725942
 ⋮
 4.0152999491485
 4.09211273193741
 4.180796019940446
 4.2730247630650195
 4.380197444098506
 4.508072668880143
 4.6730771771992465
 4.824672365072814
 5.0
u: 103-element Vector{Vector{Float64}}:
 [0.0, 0.0]
 [-2.498708625494845e-7, -0.004996126168620006]
 [-3.0078506539192585e-5, -0.05453267870039511]
 [-0.0007308693648124558, -0.26274706545050436]
 [-0.0036486629493644998, -0.5658116639156922]
 [-0.009773939753921395, -0.8852830312184409]
 [-0.02133602079845812, -1.232452668334619]
 [-0.039446424252848826, -1.5584520591464437]
 [-0.06578526346696698, -1.8404767724005746]
 [-0.09989712389165716, -2.0417645099910597]
 ⋮
 [-0.9999577259905861, -0.0005348907418043129]
 [-0.9999878223871

### Saving into csv 

In [ ]:
using CSV, Tables

function save_solution_to_csv(sol, filename, joint_names, vms_compiled; rate=nothing, time_vector=nothing)
    # Generate time vector based on rate or use the provided one
    if time_vector === nothing
        if rate === nothing
            error("Either 'rate' or 'time_vector' must be provided.")
        end
        t_min, t_max = sol.t[begin], sol.t[end]
        time_vector = collect(t_min:1/rate:t_max)  # Uniform sampling
    end

    # Prepare header (positions and velocities)
    pos_names = [joint_names[i] * "_pos" for i in 1:length(joint_names)]
    vel_names = [joint_names[i] * "_vel" for i in 1:length(joint_names)]
    header = ["time"; pos_names; vel_names]

    # Initialize data array with time
    data = time_vector
    pos_data = []
    vel_data = []

    n = length(joint_names)

    # Loop through each joint and collect its values at all time steps
    for joint_name in joint_names
        # Get the joint ID from the compiled robot
        jointID = get_compiled_jointID(vms_compiled, ".robot." * joint_name)
        joint = vms_compiled[jointID]

        # Get the indices for position and velocity
        joint_index = q_idxs(joint)[1]

        joint_positions = [sol(t)[joint_index] for t in time_vector]  # Positions
        joint_velocities = [sol(t)[n + joint_index] for t in time_vector]  # Velocities

        # First iteration 
        if length(pos_data) == 0
            pos_data = joint_positions
            vel_data = joint_velocities
        else
            pos_data = hcat(pos_data, joint_positions)
            vel_data = hcat(vel_data,joint_velocities)
        end
    end

    data = hcat(data, pos_data, vel_data)

    CSV.write(filename, Tables.table(data); header=header)

    println("CSV saved to $filename")
end


joint_names = collect(keys(joints(vms.robot)))
save_solution_to_csv(sol, "two_joints_solution.csv" , joint_names, vms_compiled; rate=10)
